In [2]:
def fetch_field_data(cursor, field_name):
    """
    Fetches tender_id and specified field data from the database.

    Parameters:
    - cursor: A database cursor object.
    - field_name: The name of the field to fetch data from.

    Returns:
    A list of dictionaries, each containing 'tender_id' and the specified field data.
    """
    

    
    # 构造SQL查询语句
    query = f"SELECT tender_id, {field_name} FROM tender_detail"
    
    # 执行查询
    try:
        cursor.execute(query)
        # 获取所有记录
        rows = cursor.fetchall()
        # 转换记录为JSON格式
        data = [{"tender_id": row[0], field_name: row[1]} for row in rows]
        return data
    except Exception as e:
        print(f"Error fetching data: {e}")
        return []


In [4]:
from database_util import connect_db
db = connect_db()
cursor = db.cursor()

# 示例使用
# 假设您想提取 'chief_supervisor_qualification_requirement' 字段的数据
field_name = 'chief_supervisor_qualification_requirement'
data = fetch_field_data(cursor, field_name)
print(data)


[{'tender_id': 1, 'chief_supervisor_qualification_requirement': '路桥相关专业高级工程师，至少担任过2项类似工程土建工程施工监理的监理负责人。'}, {'tender_id': 3, 'chief_supervisor_qualification_requirement': '具备国家注册监理工程师执业资格（市政公用工程专业）'}, {'tender_id': 4, 'chief_supervisor_qualification_requirement': '具备国家注册监理工程师执业资格（市政公用工程专业）'}, {'tender_id': 6, 'chief_supervisor_qualification_requirement': '拟派本工程总监理工程师具备市政公用工程专业和建筑工程专业的注册监理工程师证书'}, {'tender_id': 7, 'chief_supervisor_qualification_requirement': '具有建设部2006年4月1日后颁发的中华人民共和国注册监理工程师注册执业证书，专业为市政公用工程，注册执业单位为投标人本公司，且在有效期内'}, {'tender_id': 8, 'chief_supervisor_qualification_requirement': '1、需登记在东莞市公共资源交易企业库中；2、具备建设部核发的《中华人民共和国注册监理工程师注册执业证书》，上述证书上注明的注册专业为房屋建筑工程专业，上述证书上注明的注册执业单位（或工作单位）为投标人本单位；3、项目总监理工程师不得同时在除本工程外的其他任何工程项目中任职。'}, {'tender_id': 9, 'chief_supervisor_qualification_requirement': '具有市政公用工程专业国家注册监理工程师资格'}, {'tender_id': 10, 'chief_supervisor_qualification_requirement': '房屋建筑工程'}, {'tender_id': 11, 'chief_supervisor_qualification_requirement': '项目总监必须具备国家注册监理工程师执业资格'}, {'ten

In [63]:
from zhipuai import ZhipuAI

client = ZhipuAI(api_key="e6af334544b37a85e83900a9152eb9a0.GzjnBTUhPOhDDJhx")

def proofread_values(values):
    tools = [{
        "type": "function",
        "function": {
            "name": "insert_data",
            "description": "将注册资格证书专业ID、职称等级ID、最低学历要求ID、总监相关业绩要求和总监同时任职项目数量限制等信息插入到数据库中。",
            "parameters": {
            "type": "object",
            "properties": {
                "tender_id": {
                "description": "tender_id",
                "type": "string"
                },
                "专业ID": {
                "description": "注册资格证书专业ID",
                "type": "string"
                },
                "职称等级ID": {
                "description": "职称等级ID",
                "type": "string"
                },
                "学历ID": {
                "description": "最低学历要求ID",
                "type": "string"
                },
                "相关业绩要求": {
                "description": "总监相关业绩要求，例如‘至少担任过2项类似工程的监理负责人’。",
                "type": "number"
                },
                "同时任职项目数": {
                "description": "总监同时任职项目数量限制，例如‘在任职期间能参与的其他在施项目不得超过2个’。",
                "type": "number"
                }
            },
            "required": ["专业ID", "职称等级ID", "学历ID", "相关业绩要求", "同时任职项目数"]
            }
        }
        }]


    # 清空对话
    messages = []
# 
    messages.append({"role": "system", "content": """
                        从招标公告中的总监要求信息中，根据预定义的类别和标识符提取具体信息，通过insert_data将数据插入到数据库中。arguments里如果没有相应的信息，则输出“Null”。

                        需要提取的信息:
                        1. 注册资格证书专业ID: `专业ID`
                        2. 职称等级ID: `职称等级ID`
                        3. 最低学历要求ID: `学历ID`
                        4. 总监相关业绩要求: `相关业绩要求`项（例如，“至少担任过2项类似工程的监理负责人”。）
                        5. 总监同时任职项目数量限制: `最多同时任职项目数`项（例如，“在任职期间能参与的其他在施项目不得超过2个”，若不的任职任何其他项目则‘最多0个’。）

                        预定义的标识符:
                        - 注册资格证书专业及对应ID:
                        房屋建筑工程:1; 冶炼工程:2; 矿山工程:3; 化工石油工程:4; 水利水电工程:5; 电力工程:6; 农林工程:7; 铁路工程:8; 公路工程:9; 港口与航道工程:10; 航天航空工程:11; 通信工程:12; 市政公用工程:13; 机电安装工程:14; 道路与桥梁:15; 隧道工程:16; 公路机电工程:17; 港口工程:18; 航道工程:19; 水运机电工程:20; 水利工程施工监理:21; 水土保持工程施工监理:22; 机电及金属结构设备制造监理:23; 水利工程建设环境保护监理:24;

                        - 职称等级及对应ID:
                        教授级高级工程师（正高级）:1 ;高级工程师（副高级）:2;工程师（中级）:3;助理工程师（初级）:4

                        - 学历及对应ID:
                        中专/高中:1;专科:2,本科:3;硕士研究生:4;博士研究生:5;其他:6

                        提取过程:
                        1. 从招标信息字段中读取总监要求信息。
                        2. 将要求信息与预定义的专业、职称等级和最低学历标识符进行匹配，可根据语意进行合理推断。
                        3. 提取所需的类似项目经验数量和可同时任职的项目限制数量。
                        4. 输出提取的ID及要求信息。如果任何信息缺失，该类别输出“Null”。

                        -----

                        示例使用:

                        输入:
                        “招标要求注册总监须具备房屋建筑工程专业资格，至少为高级工程师职称，持有硕士学位，曾至少监理过3项类似工程项目，且在任职期间最多可同时参与2个在施项目。”

                        输出:
                        1. 注册资格证书专业ID: 1
                        2. 职称等级ID: 2
                        3. 学历ID: 4
                        4. 总监相关业绩要求: 3项
                        5. 总监同时任职项目数量限制: 2项                    
                        """})

    # messages.append({"role": "user", "content": "以下是多个不同招标公告（不同 tender_id）中的总监要求相关字段信息，务必依据上述规则，对每一个tender_id内容逐一处理："})
    messages.append({"role": "user", "content": "以下是招标公告："})

    messages.append({"role": "user", "content": f"{values}"})

    response = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称 model="glm-4" model="glm-3-turbo"
        messages=messages,
        tools=tools,
        top_p=0.7,
        temperature=0.8,
        max_tokens=4096,
    )
    return response
    # return response.choices[0].message.tool_calls[0].function.arguments

In [177]:
from zhipuai import ZhipuAI

client = ZhipuAI(api_key="e6af334544b37a85e83900a9152eb9a0.GzjnBTUhPOhDDJhx")

def proofread_values(values):
    tools = [{
        "type": "function",
        "function": {
            "name": "insert_chief_supervisor_req",
            "description": "将注册资格证书专业ID(注意，如果有两个专业的要求，则分别填入两个专业id arguments)、职称等级ID、最低学历要求ID、总监相关业绩要求和总监同时任职项目数量限制等信息插入到数据库中。",
            "parameters": {
            "type": "object",
            "properties": {
                "tender_id": {
                "description": "tender_id",
                "type": "string"
                },
                "专业ID一": {
                "description": "注册资格证书专业ID:房屋建筑工程:1; 冶炼工程:2; 矿山工程:3; 化工石油工程:4; 水利水电工程:5; 电力工程:6; 农林工程:7; 铁路工程:8; 公路工程:9; 港口与航道工程:10; 航天航空工程:11; 通信工程:12; 市政公用工程:13; 机电安装工程:14; 道路与桥梁:15; 隧道工程:16; 公路机电工程:17; 港口工程:18; 航道工程:19; 水运机电工程:20; 水利工程施工监理:21; 水土保持工程施工监理:22; 机电及金属结构设备制造监理:23; 水利工程建设环境保护监理:24;",
                "type": "string",
                "enum": ["Null","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24"]
                },
                "专业ID二": {
                "description": "（可选）注册资格证书专业ID:房屋建筑工程:1; 冶炼工程:2; 矿山工程:3; 化工石油工程:4; 水利水电工程:5; 电力工程:6; 农林工程:7; 铁路工程:8; 公路工程:9; 港口与航道工程:10; 航天航空工程:11; 通信工程:12; 市政公用工程:13; 机电安装工程:14; 道路与桥梁:15; 隧道工程:16; 公路机电工程:17; 港口工程:18; 航道工程:19; 水运机电工程:20; 水利工程施工监理:21; 水土保持工程施工监理:22; 机电及金属结构设备制造监理:23; 水利工程建设环境保护监理:24;",
                "type": "string",
                "enum": ["Null","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24"]
                },
                "职称等级ID": {
                "description": "职称等级ID:教授级高级工程师（正高级）:1 ;高级工程师（副高级）:2;工程师（中级）:3;助理工程师（初级）:4",
                "type": "string",
                "enum": ["Null","1", "2", "3", "4"]
                },
                "学历ID": {
                "description": "最低学历要求ID:中专/高中:1;专科:2,本科:3;硕士研究生:4;博士研究生:5;其他:Null",
                "type": "string",
                "enum": ["Null","1", "2", "3", "4", "5"]
                },
                "相关业绩要求": {
                "description": "总监相关业绩要求，例如‘至少担任过2项类似工程的监理负责人’。没有要求则是Null。",
                "type": "number"
                },
                "同时任职项目限制": {
                "description": "总监同时任职项目数量限制，例如‘在任职期间能参与的其他在施项目不得超过2个’。没有要求则是Null。",
                "type": "number"
                }
            },
            "required": ["tender_id","专业ID一","专业ID二", "职称等级ID", "学历ID", "相关业绩要求", "同时任职项目数"]
            }
        }
        }]


    # 清空对话
    messages = []
# 
    messages.append({"role": "system", "content": """
                        从招标公告中的总监要求信息中，根据预定义的类别和标识符提取具体信息（可根据语意进行合理推断），再通过insert_chief_supervisor_req将数据插入到数据库中。arguments里如果没有相应的信息，则输出“Null”。
                        """})

    messages.append({"role": "user", "content": "以下是招标公告："})

    messages.append({"role": "user", "content": f"{values}"})

    response = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称 model="glm-4"  or "glm-3-turbo"
        messages=messages,
        tools=tools,
        top_p=0.7,
        temperature=0.8,
        max_tokens=4096,
    )
    return response
    # return response.choices[0].message.tool_calls[0].function.arguments

In [172]:
from zhipuai import ZhipuAI

client = ZhipuAI(api_key="e6af334544b37a85e83900a9152eb9a0.GzjnBTUhPOhDDJhx")

def proofread_values(values):
    tools = [{
        "type": "function",
        "function": {
            "name": "insert_data",
            "description": """将注册资格证书专业ID(注意，如果有两个专业的要求，则分别填入两个专业id一和二)、职称等级ID、最低学历要求ID、总监相关业绩要求和总监同时任职项目数量限制等信息插入到数据库中。 预定义的标识符:
                        注：如果包含两个tender_id的信息，需要分别填写两个tender_id的信息。
                        
                        - 注册资格证书专业及对应ID:
                        房屋建筑工程:1; 冶炼工程:2; 矿山工程:3; 化工石油工程:4; 水利水电工程:5; 电力工程:6; 农林工程:7; 铁路工程:8; 公路工程:9; 港口与航道工程:10; 航天航空工程:11; 通信工程:12; 市政公用工程:13; 机电安装工程:14; 道路与桥梁:15; 隧道工程:16; 公路机电工程:17; 港口工程:18; 航道工程:19; 水运机电工程:20; 水利工程施工监理:21; 水土保持工程施工监理:22; 机电及金属结构设备制造监理:23; 水利工程建设环境保护监理:24;

                        - 职称等级及对应ID:
                        教授级高级工程师（正高级）:1 ;高级工程师（副高级）:2;工程师（中级）:3;助理工程师（初级）:4

                        - 学历及对应ID:
                        中专/高中:1;专科:2,本科:3;硕士研究生:4;博士研究生:5;
                        """,
            "parameters": {
            "type": "object",
            "properties": {
                "tender_id": {
                "description": "tender_id",
                "type": "string"
                },
                "专业ID一": {
                "description": "注册资格证书专业ID一",
                "type": "string",
                "enum": ["Null","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24"]
                },
                "专业ID二": {
                "description": "（可选）注册资格证书专业ID二",
                "type": "string",
                "enum": ["Null","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24"]
                },
                "职称等级ID": {
                "description": "职称等级ID",
                "type": "string",
                "enum": ["Null","1", "2", "3", "4"]
                },
                "学历ID": {
                "description": "最低学历要求ID",
                "type": "string",
                "enum": ["Null","1", "2", "3", "4", "5"]
                },
                "相关业绩要求": {
                "description": "总监相关业绩要求，没有要求则是Null。",
                "type": "number"
                },
                "同时任职项目限制": {
                "description": "总监同时任职项目数量限制，没有要求则是Null。",
                "type": "number"
                },
                "tender_id_2": {
                "description": "tender_id",
                "type": "string"
                },
                "专业ID一_2": {
                "description": "注册资格证书专业ID一",
                "type": "string",
                "enum": ["Null","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24"]
                },
                "专业ID二_2": {
                "description": "（可选）注册资格证书专业ID二",
                "type": "string",
                "enum": ["Null","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24"]
                },
                "职称等级ID_2": {
                "description": "职称等级ID",
                "type": "string",
                "enum": ["Null","1", "2", "3", "4"]
                },
                "学历ID_2": {
                "description": "最低学历要求ID",
                "type": "string",
                "enum": ["Null","1", "2", "3", "4", "5"]
                },
                "相关业绩要求_2": {
                "description": "总监相关业绩要求，没有要求则是Null。",
                "type": "number"
                },
                "同时任职项目限制_2": {
                "description": "总监同时任职项目数量限制，没有要求则是Null。",
                "type": "number"
                }

            },
            "required": ["tender_id","专业ID一","专业ID二", "职称等级ID", "学历ID", "相关业绩要求", "同时任职项目数","tender_id_2","专业ID一_2","专业ID二_2", "职称等级ID_2", "学历ID_2", "相关业绩要求_2", "同时任职项目数_2"]
            }
        }
        }]


    # 清空对话
    messages = []
# 
    messages.append({"role": "system", "content": """
                        从招标公告中的总监要求信息中，根据预定义的类别和标识符提取具体信息（可根据语意进行合理推断），再通过insert_data将数据插入到数据库中。arguments里如果没有相应的信息，则输出“Null”。
                        """})

    messages.append({"role": "user", "content": "以下是招标公告："})

    messages.append({"role": "user", "content": f"{values}"})

    response = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称 model="glm-4"  or "glm-3-turbo"
        messages=messages,
        tools=tools,
        top_p=0.7,
        temperature=0.8,
        max_tokens=4096,
    )
    return response
    # return response.choices[0].message.tool_calls[0].function.arguments

In [168]:
data[10:12]

[{'tender_id': 16,
  'chief_supervisor_qualification_requirement': '拟派总监理工程师为市政公用工程专业注册监理工程师，持有国家住建部印发的有效注册证书，且在施项目不超过2个'},
 {'tender_id': 17,
  'chief_supervisor_qualification_requirement': '担任本工程的总监理工程师必须具有高级工程师职称证书及相应监理资格证书'}]

In [181]:
results = proofread_values(data[12])

In [182]:
print(results.choices[0].message.content)

None


In [183]:
results.choices[0].message.tool_calls[0].function.arguments

'{"tender_id":"18","专业ID一":"5","专业ID二":"13","同时任职项目限制":2,"学历ID":"Null","相关业绩要求":"Null","职称等级ID":"Null"}'

In [184]:
results.usage.model_dump()

{'prompt_tokens': 1235, 'completion_tokens': 56, 'total_tokens': 1291}

In [ ]:
def main_process(cursor):
    # 1. 从数据库获取所有招标详情
    tender_details = fetch_all_tender_details(cursor)

    # 遍历每一个招标记录
    for tender_detail in tender_details:
        tender_id = tender_detail['tender_id']
        extracted_infos = []

        # 2. 对于每一个字段，根据字段名称使用不同的提取策略
        # 假设 tender_detail 字典中已经包含了所有你感兴趣的字段及其值
        for field_name, field_value in tender_detail.items():
            if field_name == 'tender_id':
                continue  # 跳过 tender_id 字段，因为它不需要提取信息
            extracted_info = extract_info_with_tool(tender_id, field_name, field_value)
            if extracted_info:
                extracted_infos.append(extracted_info)
        
        # 3. 合并提取的信息
        combined_info = combine_info(tender_id, extracted_infos)

        # 4. 将合并后的信息插入到数据库中
        insert_into_database(cursor, combined_info)
    
    # 确保所有更改都被提交到数据库
    db.commit()
